In [6]:
# ============================================================
# SCRIPT COMPLETO (REFATORADO DO SEU NOTEBOOK exp_ml_006.ipynb)
# Gera: tabela_previsoes_xgb.csv
# ============================================================

# -----------------------------
# 0) IMPORTS
# -----------------------------

# pandas: leitura e manipulação do dataset em formato de tabela (DataFrame)
import pandas as pd

# numpy: operações numéricas auxiliares
import numpy as np

# scikit-learn: pré-processamento e pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report
)

# xgboost: modelo
from xgboost import XGBClassifier


# -----------------------------
# 1) CONFIGURAÇÕES (AJUSTE AQUI)
# -----------------------------

# caminho do seu dataset (CSV)
# - se estiver na mesma pasta do script, basta deixar o nome do arquivo
CAMINHO_DATASET = "australia_clima_v8.csv"

# nome do arquivo CSV final (saída) para o Power BI
CAMINHO_SAIDA = "tabela_previsoes_xgb.csv"

# nome da coluna alvo (target) - exatamente como no seu notebook
TARGET = "chove_amanha_vtr"

# colunas que você removeu como "irrelevantes" no seu notebook
# - "data" não entra no modelo porque é usada para split temporal e para BI
# - "choveu_hoje_fex" você removeu no seu código
COLUNAS_PARA_REMOVER = ["data", "choveu_hoje_fex"]

# proporção temporal do split (80% treino, 20% teste)
PROPORCAO_TREINO = 0.8

# threshold padrão para transformar probabilidade em classe (0/1)
THRESHOLD = 0.5

# “apelido” da versão do modelo (para rastreabilidade no Power BI)
MODELO_NOME = "xgb_refatorado_v1"


# -----------------------------
# 2) CARREGAR O DATASET (IGUAL SEU NOTEBOOK, SÓ QUE PORTÁVEL)
# -----------------------------

# df recebe o dataset carregado do CSV
df = pd.read_csv(r'C:\Users\JacyzinGuilherme(Bip\mentoria-bip\dados_editados\australia_clima_v8.csv', sep=',')

# -----------------------------
# 3) CONVERTER COLUNA "data" E ORDENAR (SPLIT TEMPORAL)
# -----------------------------

# converte a coluna 'data' para datetime, transformando textos em datas reais
# - errors="coerce" converte valores inválidos em NaT (nulo de datetime)
df["data"] = pd.to_datetime(df["data"], errors="coerce")

# remove linhas onde a 'data' ficou inválida (NaT),
# porque sem data não existe split temporal honesto
df = df.dropna(subset=["data"]).copy()

# ordena do mais antigo para o mais recente (igual seu notebook)
df = df.sort_values(by="data").reset_index(drop=True)


# -----------------------------
# 4) DEFINIR TARGET E PREPARAR DF DE MODELAGEM (IGUAL SEU FLUXO)
# -----------------------------

# remove colunas irrelevantes (data e choveu_hoje_fex) do dataframe de modelagem
# - errors="ignore" evita quebrar se uma coluna não existir por algum motivo
df_modelo = df.drop(columns=COLUNAS_PARA_REMOVER, errors="ignore").copy()

# garante que o target exista
if TARGET not in df_modelo.columns:
    raise ValueError(
        f"A coluna target '{TARGET}' não foi encontrada no dataset. "
        f"Colunas disponíveis: {list(df_modelo.columns)}"
    )

# remove linhas onde o target esteja nulo (não dá para treinar nem avaliar)
df_modelo = df_modelo.dropna(subset=[TARGET]).copy()

# força o target a ser inteiro 0/1
df_modelo[TARGET] = df_modelo[TARGET].astype(int)

# separa X (features) e y (target)
X = df_modelo.drop(columns=[TARGET], errors="ignore")
y = df_modelo[TARGET]


# -----------------------------
# 5) REMOVER TODAS AS COLUNAS *_isna (IGUAL SEU NOTEBOOK)
# -----------------------------

# identifica colunas que terminam com "_isna"
colunas_isna = [c for c in X.columns if c.endswith("_isna")]

# remove essas colunas do X
X = X.drop(columns=colunas_isna, errors="ignore").copy()

# remove essas colunas também do df_modelo (para manter consistência)
df_modelo = df_modelo.drop(columns=colunas_isna, errors="ignore").copy()


# -----------------------------
# 6) SPLIT TEMPORAL 80/20 POR ÍNDICE (IGUAL SEU NOTEBOOK)
# -----------------------------

# total_linhas recebe o total de linhas do df_modelo
total_linhas = len(df_modelo)

# indice_corte separa treino e teste (80% do começo para treino)
indice_corte = int(total_linhas * PROPORCAO_TREINO)

# X_train pega do início até o corte
X_train = X.iloc[:indice_corte].copy()

# X_test pega do corte até o final
X_test = X.iloc[indice_corte:].copy()

# y_train pega do início até o corte
y_train = y.iloc[:indice_corte].copy()

# y_test pega do corte até o final
y_test = y.iloc[indice_corte:].copy()

# também vamos guardar as datas correspondentes ao teste para exportar na tabela final
# atenção: como removemos 'data' de df_modelo, pegamos da base original df (ordenada)
# mas precisamos alinhar índices:
# - como df_modelo veio de df com drop de colunas, a ordem e o index são os mesmos
data_test = df.iloc[indice_corte:]["data"].copy()

# também precisamos da localidade para BI (se existir)
# - se não existir, a gente cria uma coluna “desconhecida”
if "localidade" in df.columns:
    localidade_test = df.iloc[indice_corte:]["localidade"].astype(str).copy()
else:
    localidade_test = pd.Series(["desconhecida"] * len(X_test))


# -----------------------------
# 7) SEPARAR COLUNAS NUMÉRICAS E CATEGÓRICAS (IGUAL SEU NOTEBOOK)
# -----------------------------

# colunas categóricas: object ou category
colunas_categoricas = X_train.select_dtypes(include=["object", "category"]).columns.tolist()

# colunas numéricas: int64/float64 (ou int32/float32 também, por segurança)
colunas_numericas = X_train.select_dtypes(include=["int64", "float64", "int32", "float32"]).columns.tolist()


# -----------------------------
# 8) PREPROCESSADOR IGUAL O SEU "preprocessor_rf"
#    - OneHot nas categóricas
#    - passthrough nas numéricas
# -----------------------------

# OneHotEncoder:
# - handle_unknown="ignore": se aparecer categoria nova no teste, não quebra
# - sparse_output=False: retorna matriz densa (igual seu notebook)
transformador_categorico = OneHotEncoder(
    handle_unknown="ignore",
    sparse_output=False
)

# ColumnTransformer:
# - aplica onehot nas categóricas
# - mantém numéricas como estão ("passthrough")
preprocessador = ColumnTransformer(
    transformers=[
        ("cat", transformador_categorico, colunas_categoricas),
        ("num", "passthrough", colunas_numericas),
    ],
    remainder="drop"
)


# -----------------------------
# 9) SCALE_POS_WEIGHT (IGUAL SEU NOTEBOOK)
# -----------------------------

# n0: quantidade de classe 0 no treino
n0 = int((y_train == 0).sum())

# n1: quantidade de classe 1 no treino
n1 = int((y_train == 1).sum())

# evita divisão por zero (caso extremamente improvável)
if n1 == 0:
    scale_pos_weight = 1.0
else:
    scale_pos_weight = n0 / n1


# -----------------------------
# 10) MODELO XGBOOST (ALINHADO AO SEU NOTEBOOK)
# -----------------------------
# Aqui eu vou manter o núcleo do seu modelo:
# - n_estimators
# - learning_rate
# - max_depth
# - subsample
# - colsample_bytree
# - reg_lambda
# - objective
# - eval_metric="aucpr" (como no seu)
# - scale_pos_weight (como no seu)
# - random_state
# - n_jobs

modelo_xgb = XGBClassifier(
    n_estimators=500,
    learning_rate=0.04,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    objective="binary:logistic",
    eval_metric="aucpr",
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1
)

# pipeline final:
# 1) preprocessa (onehot + passthrough)
# 2) treina XGBoost
pipeline_xgb = Pipeline(
    steps=[
        ("preprocessamento", preprocessador),
        ("modelo", modelo_xgb)
    ]
)


# -----------------------------
# 11) TREINAR (SOMENTE NO TREINO)
# -----------------------------

# fit aprende:
# - mapeamento do one-hot (categorias existentes no treino)
# - e então treina o XGBoost no espaço transformado
pipeline_xgb.fit(X_train, y_train)


# -----------------------------
# 12) PREVER NO TESTE (FUTURO)
# -----------------------------

# probabilidade de classe 1 (chuva)
y_proba_xgb = pipeline_xgb.predict_proba(X_test)[:, 1]

# classe prevista aplicando threshold
y_pred_xgb = (y_proba_xgb >= THRESHOLD).astype(int)


# -----------------------------
# 13) (OPCIONAL) IMPRIMIR MÉTRICAS PARA CONFERÊNCIA
# -----------------------------

print("========== MÉTRICAS (TESTE 20% FINAL) ==========")
print(f"Total linhas: {total_linhas}")
print(f"Índice corte (80%): {indice_corte}")
print(f"Treino: {X_train.shape} | Teste: {X_test.shape}")
print(f"scale_pos_weight = {scale_pos_weight:.4f}")
print(f"threshold = {THRESHOLD}")

print("\nMatriz de confusão:")
print(confusion_matrix(y_test, y_pred_xgb))

print("\nClassification report:")
print(classification_report(y_test, y_pred_xgb, digits=4))

print("\nAcurácia:", f"{accuracy_score(y_test, y_pred_xgb)*100:.2f}%")
print("Precisão:", f"{precision_score(y_test, y_pred_xgb, zero_division=0)*100:.2f}%")
print("Recall:", f"{recall_score(y_test, y_pred_xgb, zero_division=0)*100:.2f}%")
print("F1:", f"{f1_score(y_test, y_pred_xgb, zero_division=0)*100:.2f}%")

# ROC-AUC pode falhar se o y_test tiver só uma classe (raro)
try:
    print("ROC-AUC:", f"{roc_auc_score(y_test, y_proba_xgb):.4f}")
except Exception as e:
    print("ROC-AUC: não calculado (motivo:", str(e), ")")


# -----------------------------
# 14) MONTAR A TABELA FINAL PARA O POWER BI
# -----------------------------

# cutoff_date: primeira data do conjunto de teste
cutoff_date = pd.to_datetime(data_test.min())

# train_end_date: última data do conjunto de treino
train_end_date = pd.to_datetime(df.iloc[:indice_corte]["data"].max())

# test_start_date: primeira data do conjunto de teste (igual cutoff_date na prática)
test_start_date = pd.to_datetime(data_test.min())

# tabela_previsoes_xgb: dataframe final que o Power BI vai consumir
tabela_previsoes_xgb = pd.DataFrame({
    # data: a data real de cada linha prevista (teste)
    "data": pd.to_datetime(data_test).dt.date,

    # localidade: a localidade correspondente (se existir)
    "localidade": localidade_test.values,

    # y_true: verdade (o que realmente aconteceu)
    "y_true": y_test.values.astype(int),

    # y_pred: classe prevista pelo modelo (0/1)
    "y_pred": y_pred_xgb.astype(int),

    # y_proba: probabilidade do modelo para classe 1 (chuva)
    "y_proba": y_proba_xgb.astype(float),

    # metadados úteis para BI / rastreabilidade
    "modelo": MODELO_NOME,
    "threshold": THRESHOLD,
    "cutoff_date": cutoff_date.date(),
    "train_end_date": train_end_date.date(),
    "test_start_date": test_start_date.date()
})

# ordena para ficar “bonito” e consistente no BI
tabela_previsoes_xgb = tabela_previsoes_xgb.sort_values(by=["data", "localidade"]).reset_index(drop=True)


# -----------------------------
# 15) EXPORTAR CSV FINAL
# -----------------------------

tabela_previsoes_xgb.to_csv(r'C:\Users\JacyzinGuilherme(Bip\mentoria-bip\dados_editados\tabela_previsoes_xgb.csv', index=False, encoding="utf-8")

print("\n✅ ARQUIVO GERADO COM SUCESSO!")
print("Arquivo:", CAMINHO_SAIDA)
print("Linhas:", len(tabela_previsoes_xgb))
print("Colunas:", tabela_previsoes_xgb.shape[1])


========== MÉTRICAS (TESTE 20% FINAL) ==========
Total linhas: 50159
Índice corte (80%): 40127
Treino: (40127, 37) | Teste: (10032, 37)
scale_pos_weight = 3.6784
threshold = 0.5

Matriz de confusão:
[[6750 1138]
 [ 494 1650]]

Classification report:
              precision    recall  f1-score   support

           0     0.9318    0.8557    0.8921      7888
           1     0.5918    0.7696    0.6691      2144

    accuracy                         0.8373     10032
   macro avg     0.7618    0.8127    0.7806     10032
weighted avg     0.8591    0.8373    0.8445     10032


Acurácia: 83.73%
Precisão: 59.18%
Recall: 76.96%
F1: 66.91%
ROC-AUC: 0.8996

✅ ARQUIVO GERADO COM SUCESSO!
Arquivo: tabela_previsoes_xgb.csv
Linhas: 10032
Colunas: 10
